# AIX System Metrics Analysis & Model Training

This notebook connects to the PostgreSQL database to retrieve system metrics data collected from AIX servers. We'll use this data to:
1. Explore and visualize the metrics,
2. Preprocess the data for machine learning,
3. Train anomaly detection models,
4. Evaluate and save the models for use in the Django application

In [1]:
# %% [markdown]
# # Hybrid Anomaly Detection System
# ## Multi-file Data Integration, Model Training & Evaluation
# 
# This notebook:
# 1. Loads vmstat, iostat, netstat, and process CSV files
# 2. Integrates them into a unified dataset
# 3. Trains a hybrid anomaly detection model
# 4. Evaluates performance with comprehensive metrics
# 5. Visualizes confusion matrix and other key metrics

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import (
    confusion_matrix, 
    classification_report, 
    f1_score, 
    precision_recall_curve,
    auc,
    roc_curve,
    roc_auc_score,
    precision_score,
    recall_score,
    accuracy_score
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras import backend as K
import joblib
import tensorflow as tf
import pywt
from sklearn.decomposition import NMF
import gc



ModuleNotFoundError: No module named 'tensorflow'